In [1]:
using BenchmarkTools
using Printf
using JLD2

include("Chebyshev.jl")
using .Chebyshev

In [2]:
@load "l1domains.jld2" L₁1 L₁2 L₁3
@load "l1points.jld2" xv L ∇L HL

L₁c = ChebyshevCluster(L₁1, L₁2, L₁3);

In [3]:
Lc = Vector{Float64}(undef, size(xv, 1))
∇Lc = Matrix{Float64}(undef, size(xv))
HLc = Array{Float64, 3}(undef, size(xv)..., 3);

In [4]:
i = 1
Lc1 = L₁c(view(xv, i, :))
Lc2, ∇Lc1 = gradient(L₁c, view(xv, i, :))
Lc3, ∇Lc2, HL1 = hessian(L₁c, view(xv, i, :));

In [5]:
for i in axes(xv, 1)
    Lc1 = L₁c(view(xv, i, :))
    Lc2, ∇Lc1 = gradient(L₁c, view(xv, i, :))
    Lc3, ∇Lc2, HL1 = hessian(L₁c, view(xv, i, :))

    if Lc1 ≈ Lc2 && Lc1 ≈ Lc3
        Lc[i] = Lc1
    end

    if ∇Lc1 ≈ ∇Lc2
        ∇Lc[i, :] = ∇Lc1
    end
    
    HLc[i, :, :] = HL1
end;

In [6]:
err_L = @. abs(L - Lc)
err_∇L = @. abs(∇L - ∇Lc)
err_HL = @. abs(HL - HLc)

mean_err_L = mean(err_L)
mean_err_∇L = dropdims(mean(err_∇L; dims=1); dims=1)
mean_err_HL = dropdims(mean(err_HL; dims=1); dims=1)

std_err_L = std(err_L)
std_err_∇L = dropdims(std(err_∇L; dims=1); dims=1)
std_err_HL = dropdims(std(err_HL; dims=1); dims=1)

lim_err_L = mean_err_L + 3 * std_err_L
lim_err_∇L = reshape(mean_err_∇L + 3 * std_err_∇L, 1, 3)
lim_err_HL = reshape(mean_err_HL + 3 * std_err_HL, 1, 3, 3)

pct_err_L = sum(err_L .< lim_err_L) * 100 / size(xv, 1)
pct_err_∇L = dropdims(count(err_∇L .< lim_err_∇L; dims=1); dims=1) * 100 / size(xv, 1)
pct_err_HL = dropdims(sum(err_HL .< lim_err_HL; dims=1); dims=1) * 100 / size(xv, 1);

In [7]:
result = """
L₁c mean absolute error = $(@sprintf("%.0e", mean_err_L))

∇L₁c mean absolute error = $(@sprintf("[%.0e %6.0e %6.0e]", mean_err_∇L...))

HL₁c mean absolute error = $(@sprintf("[%.0e %6.0e %6.0e]", mean_err_HL[1, :]...))
                           $(@sprintf("[%.0e %6.0e %6.0e]", mean_err_HL[2, :]...))
                           $(@sprintf("[%.0e %6.0e %6.0e]", mean_err_HL[3, :]...))

For L₁c, ∇L₁c and HL₁c, the error value for each point is compared with a corresponding reference
value. The reference value is the mean absolute error plus three times the standard deviation. The
following results display the percentage of points for which the error is less than the reference.

Percentage of L₁c error values lower than reference = $(@sprintf("%.1f%%", pct_err_L))

Percentage of ∇L₁c error values lower than reference = $(@sprintf("[%.1f%% %5.1f%% %5.1f%%]", pct_err_∇L...))

Percentage of HL₁c error values lower than reference = $(@sprintf("[%.1f%% %5.1f%% %5.1f%%]", pct_err_HL[1, :]...))
                                                       $(@sprintf("[%.1f%% %5.1f%% %5.1f%%]", pct_err_HL[2, :]...))
                                                       $(@sprintf("[%.1f%% %5.1f%% %5.1f%%]", pct_err_HL[3, :]...))
"""

println(result)

L₁c mean absolute error = 1e-15

∇L₁c mean absolute error = [3e-14  1e-14  2e-14]

HL₁c mean absolute error = [3e-12  7e-13  9e-13]
                           [7e-13  1e-12  7e-13]
                           [9e-13  7e-13  3e-12]

For L₁c, ∇L₁c and HL₁c, the error value for each point is compared with a corresponding reference
value. The reference value is the mean absolute error plus three times the standard deviation. The
following results display the percentage of points for which the error is less than the reference.

Percentage of L₁c error values lower than reference = 99.2%

Percentage of ∇L₁c error values lower than reference = [98.6%  98.6%  99.5%]

Percentage of HL₁c error values lower than reference = [98.8%  98.9%  99.5%]
                                                       [98.9%  99.0%  99.1%]
                                                       [99.5%  99.1%  99.7%]

